## Preparación
- Asegúrate de que este notebook vive junto al archivo `faq_grupo_lazarus.xlsx`.
- Ejecuta cada celda (Shift + Enter) antes de pasar a la siguiente.
- Si trabajas en Codespaces, confirma que la carpeta del repositorio está montada.

In [6]:
import pandas as pd

## E - Extract (Carga de datos)
Revisamos la estructura del conjunto de datos y detectamos problemas de integridad.

In [12]:
DATA_PATH = 'faq_grupo_lazarus.xlsx'
df = pd.read_excel(DATA_PATH)
df.head()

,Pregunta,Respuesta,Categoría,Limpiar,Razón para la Limpieza (Escenario de Práctica)
0,¿Qué es ADMIX IM-1?,Es un producto para solucionar problemas de hu...,Productos,No,Correctamente formateado.
1,¿Dónde se encuentra la sede principal de Lazarus?,La Sede Corporativa y Principal está en San Pe...,Ubicaciones,No,Correctamente formateado.
2,¿Cuál es el horario de atención?,Nuestro horario es Lunes a Viernes 7:30 AM - 4...,Contacto,No,Correctamente formateado.
3,¿Qué son los aditivos que usamos?,Aditivos químicos especializados para la const...,Productos,Sí,Inconsistencia de Caso (Pregunta): La pregunta...
4,¿Puedo solicitar crédito para mis compras?,NaN,Servicios,Sí,Valores Nulos (NaN): La respuesta está vacía (...


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 5 columns):
 #   Column                                          Non-Null Count  Dtype 
---  ------                                          --------------  ----- 
 0   Pregunta                                        15 non-null     object
 1   Respuesta                                       13 non-null     object
 2   Categoría                                       15 non-null     object
 3   Limpiar                                         15 non-null     object
 4   Razón para la Limpieza (Escenario de Práctica)  15 non-null     object
dtypes: object(5)
memory usage: 732.0+ bytes


Observamos que la columna `RESPUESTA` tiene valores nulos. A continuación cuantificamos y corregimos los principales problemas de calidad.

## T - Transform (Limpieza)
### 1. Integridad: Valores nulos

In [10]:
df.isna().sum()

Pregunta                                          0
Respuesta                                         2
Categoría                                         0
Limpiar                                           0
Razón para la Limpieza (Escenario de Práctica)    0
dtype: int64

In [20]:
# Formatear los nombres de columnas a minúsculas y eliminar espacios en blanco por _
df.columns = df.columns.str.lower().str.strip().str.replace(' ', '_')
# eliminar acentos
df.columns = df.columns.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
df.head()

,pregunta,respuesta,categoria,limpiar,razon_para_la_limpieza_(escenario_de_practica)
0,¿Qué es ADMIX IM-1?,Es un producto para solucionar problemas de hu...,Productos,No,Correctamente formateado.
1,¿Dónde se encuentra la sede principal de Lazarus?,La Sede Corporativa y Principal está en San Pe...,Ubicaciones,No,Correctamente formateado.
2,¿Cuál es el horario de atención?,Nuestro horario es Lunes a Viernes 7:30 AM - 4...,Contacto,No,Correctamente formateado.
3,¿Qué son los aditivos que usamos?,Aditivos químicos especializados para la const...,Productos,Sí,Inconsistencia de Caso (Pregunta): La pregunta...
4,¿Puedo solicitar crédito para mis compras?,No disponible,Servicios,Sí,Valores Nulos (NaN): La respuesta está vacía (...


In [21]:
df['respuesta'] = df['respuesta'].fillna('No disponible')
df[df['respuesta'] == 'No disponible'].head()

,pregunta,respuesta,categoria,limpiar,razon_para_la_limpieza_(escenario_de_practica)
4,¿Puedo solicitar crédito para mis compras?,No disponible,Servicios,Sí,Valores Nulos (NaN): La respuesta está vacía (...
13,¿Necesito registro para la biblioteca?,No disponible,Biblioteca,Sí,Valores Nulos (NaN): Respuesta vacía. Este es ...


### 2. Consistencia: Normalización de texto
Unificamos el formato de la columna `TÓPICO`.

In [24]:
df['categoria'].unique()

array(['Productos', 'Ubicaciones', 'Contacto', 'Servicios', 'Marcas',
       'UBICACIONES', 'Testimonios', 'Biblioteca'], dtype=object)

In [25]:
df['categoria'] = df['categoria'].str.lower().str.strip()
df['categoria'].unique()

array(['productos', 'ubicaciones', 'contacto', 'servicios', 'marcas',
       'testimonios', 'biblioteca'], dtype=object)

### 3. Unicidad: Filas duplicadas

In [26]:
df.duplicated().sum()

np.int64(0)

In [28]:
# revisar duplicados en cada column
df.apply(lambda x: x.duplicated().sum())

pregunta                                           1
respuesta                                          2
categoria                                          8
limpiar                                           13
razon_para_la_limpieza_(escenario_de_practica)     4
dtype: int64

In [30]:
# eliminar el duplicado de respuestas
df = df.drop_duplicates(subset=['pregunta', 'respuesta'])
df.apply(lambda x: x.duplicated().sum())

pregunta                                           0
respuesta                                          1
categoria                                          7
limpiar                                           12
razon_para_la_limpieza_(escenario_de_practica)     4
dtype: int64

## T - Transform (Feature Engineering)
Creamos una nueva columna que captura la longitud de cada pregunta.

In [31]:
df['longitud_pregunta'] = df['pregunta'].str.len()
df[['pregunta', 'longitud_pregunta']].head()

,pregunta,longitud_pregunta
0,¿Qué es ADMIX IM-1?,19
1,¿Dónde se encuentra la sede principal de Lazarus?,49
2,¿Cuál es el horario de atención?,32
3,¿Qué son los aditivos que usamos?,33
4,¿Puedo solicitar crédito para mis compras?,42


## L - Load (Exportar)
Guardamos el resultado limpio para compartirlo o seguir procesando.

In [32]:
OUTPUT_PATH = 'faq_limpio.csv'
df.to_csv(OUTPUT_PATH, index=False)
OUTPUT_PATH

'faq_limpio.csv'

### Verificación rápida
En Codespaces puedes abrir el panel del explorador de archivos y confirmar que `faq_limpio.csv` apareció en el directorio raíz del repositorio.